In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from synth_dim_model import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import pandas as pd
import concurrent.futures

import warnings
warnings.filterwarnings("ignore")

In [2]:
def compute_values(mu_V_ratio, J_V_ratio, V, N, M, states):
    """Compute energy gap and sigma for given parameters."""
    H = construct_rescaled_hamiltonian(N, M, V, mu_V_ratio, J_V_ratio)
    eigenvalues, eigenvectors = exact_diagonalize(H)
    energy_gap = eigenvalues[1] - eigenvalues[0]
    sigma = sigma_ij(0, 1, eigenvectors[0], states, N, M) / M
    return energy_gap, sigma

N = 2
M = 11

min_J_V_ratio = -5
max_J_V_ratio = 5
max_mu_V_ratio = 5
resolution = 100

mu_V_ratios = np.linspace(0, max_mu_V_ratio, resolution)
J_V_ratios = np.linspace(min_J_V_ratio, max_J_V_ratio, resolution)
mu_V_ratio_grid, J_V_ratio_grid = np.meshgrid(mu_V_ratios, J_V_ratios)

for V in [-1,1]:
    sigma_grid = np.zeros_like(mu_V_ratio_grid)
    energy_gap_over_V_grid = np.zeros_like(mu_V_ratio_grid)
    states, _ = enumerate_states(N, M)
    
    # Use ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_index = {}
        
        for i, mu_V_ratio in enumerate(mu_V_ratios):
            for j, J_V_ratio in enumerate(J_V_ratios):
                future = executor.submit(compute_values, mu_V_ratio, J_V_ratio, V, N, M, states)
                future_to_index[future] = (j, i)
        
        for future in concurrent.futures.as_completed(future_to_index):
            j, i = future_to_index[future]
            try:
                energy_gap, sigma = future.result()
                energy_gap_over_V_grid[j, i] = energy_gap
                sigma_grid[j, i] = sigma
            except Exception as e:
                print(f"Error computing values for indices ({j}, {i}): {e}")
    
    sign = "positive" if V > 0 else "negative"
    
    data_energy = np.column_stack((mu_V_ratio_grid.ravel(), J_V_ratio_grid.ravel(), energy_gap_over_V_grid.ravel()))
    energy_filename = f"energy_gap_V_{sign}_N={N}_M={M}.csv"
    np.savetxt(energy_filename, data_energy, delimiter=",", header="mu_V_ratio,J_V_ratio,energy_gap", comments="")
    print(f"Saved energy gap data to {energy_filename}")
    
    data_sigma = np.column_stack((mu_V_ratio_grid.ravel(), J_V_ratio_grid.ravel(), sigma_grid.ravel()))
    sigma_filename = f"sigma_V_{sign}_N={N}_M={M}.csv"
    np.savetxt(sigma_filename, data_sigma, delimiter=",", header="mu_V_ratio,J_V_ratio,sigma", comments="")
    print(f"Saved sigma data to {sigma_filename}")


Saved energy gap data to energy_gap_V_negative_N=2_M=11.csv
Saved sigma data to sigma_V_negative_N=2_M=11.csv
Saved energy gap data to energy_gap_V_positive_N=2_M=11.csv
Saved sigma data to sigma_V_positive_N=2_M=11.csv
